In [ ]:
import h5py as h5;
import numpy as np;
import matplotlib.pyplot as plt;

In [ ]:
filedir = '/sdata/noise/'
#filename = 'ADCNoise_All_Terminated_FS_I0_D20200829_T130503_F0001.hdf5'
filename = 'ADCNoise_GS_I0_D20200829_T142909_F0001.hdf5'
filename = 'ADCXTalk_TrigCont_I0_D20200829_T142125_F0001.hdf5'
filename = 'ADCXTalk_TrigCont_GS_I0_D20200829_T142703_F0001.hdf5'

file = h5.File( filedir+filename )
print( file.attrs['comment'] )

ch_indices = file['adc1'].attrs['adc_channel_indices']
nb_samples = file['adc1'].attrs['nb_samples']
nb_events = file['adc1'].attrs['nb_events']
sample_rate = file['adc1'].attrs['sample_rate']
cal = file['adc1'].attrs['adc_conversion_factor']

In [ ]:
evt_index = 1

fig, axs = plt.subplots(8, figsize=(24,20))

for ch_index in np.arange(0,8):
    event_name = 'event_%d' % evt_index
    event = file['adc1'][event_name][ch_index][:1000]

    axs[ch_index].plot( event )

In [ ]:
evt_index = 1

#fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(24,20))
#axs = ax.flatten()

hist = {}
bins = np.arange(-100,101)

for ch_index in np.arange(0,8):
    event_name = 'event_%d' % evt_index
    event = file['adc1'][event_name][ch_index]
    hist[ch_index],_ = np.histogram( event, bins)
    print( '%.6f' % np.std(event) )

#axs[ch_index].plot( event )

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(24,20))
axs = ax.flatten()

for ch_index in np.arange(0,8):
    axs[ch_index].bar( (bins[1:]-0.5)[ hist[ch_index]>0 ], hist[ch_index][ hist[ch_index]>0 ], width=1 )

In [ ]:
file.close()

In [ ]:
#
# Cross talk between different channels
#

file.close()

filedir = '/sdata/noise/'
filename = 'ADCXTalk_TrigCont_GS_I0_D20200829_T142703_F0001.hdf5'
file = h5.File( filedir+filename )
print( file.attrs['comment'] )

ch_indices = file['adc1'].attrs['adc_channel_indices']
nb_samples = file['adc1'].attrs['nb_samples']
nb_events = file['adc1'].attrs['nb_events']
sample_rate = file['adc1'].attrs['sample_rate']
cal = file['adc1'].attrs['adc_conversion_factor']

In [ ]:
evt_index = 1

fig, axs = plt.subplots(8, figsize=(24,20))

for ch_index in np.arange(0,8):
    event_name = 'event_%d' % evt_index
    event = file['adc1'][event_name][ch_index][:10000]

    axs[ch_index].plot( event )

In [ ]:
psd = {}

time = np.linspace( 0, nb_samples/sample_rate, nb_samples)
freq = np.fft.fftfreq( n=t.shape[-1], d=1./sample_rate)
freq_bin = freq[1]-freq[0]

Vfreq = {}
Vf = []


fig, axs = plt.subplots(8, figsize=(24,20))


for ch_index in np.arange(0,8):
    
    counter = 0;
    
    for evt_index in range( 1, nb_events+1 ):
        event_name = 'event_%d' % evt_index
        event = file['adc1'][event_name][ch_index]
        
        if counter==0:
            Vf = ( np.abs(np.fft.fft(event))**2 ) /freq_bin/nb_samples
                # divide by frequency bin width and number of samples to normalize.
        else:
            Vf += ( np.abs(np.fft.fft(event))**2 ) /freq_bin/nb_samples
        
        counter += 1
        
    Vfreq[ch_index] = Vf/counter
    
    axs[ch_index].semilogx()
    axs[ch_index].semilogy()
    axs[ch_index].plot( freq[freq>0], Vfreq[ch_index][freq>0] )

In [ ]:
fig, axs = plt.subplots(8, figsize=(24,20))


for ch_index in np.arange(0,8):
    axs[ch_index].semilogy()
    axs[ch_index].plot( freq[freq>0][900:1100], Vfreq[ch_index][freq>0][900:1100] )
    roi = 1.0*Vfreq[ch_index][freq>0][900:1100]
    #print( '%.5e %.5e' % ( np.amax(roi), np.average(roi) + 2*np.std(roi) ) )
    print( '%.5e' % np.amax(roi) )

In [ ]:
file.close()